<a href="https://colab.research.google.com/github/mscholl96/mad-recime/blob/master/network/LSTM/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

## Basic includes

In [1]:
!pip install ipython-autotime
%load_ext autotime

time: 2.46 ms (started: 2022-02-20 15:23:52 +00:00)


In [2]:
!pip install word2vec

time: 2.63 s (started: 2022-02-20 15:23:52 +00:00)


In [3]:
import word2vec
from collections import Counter # https://pymotw.com/2/collections/counter.html

import pandas as pd
import numpy as np
import itertools
import re
import os

import glob

time: 66 ms (started: 2022-02-20 15:23:54 +00:00)


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

dataPath = '/content/drive/MyDrive/TP2/Datasets/Recipe1M/'
import sys
sys.path.append(dataPath)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
time: 1.63 s (started: 2022-02-20 15:23:54 +00:00)


## Import Data

In [5]:
TIMESTAMP = '2022_02_11'

time: 1.74 ms (started: 2022-02-20 15:23:56 +00:00)


In [6]:
baseFrame = pd.DataFrame()

smallSet = True

if(os.path.exists(dataPath + TIMESTAMP + '/recipes_valid_full.pkl')):
  baseFrame = pd.read_pickle(dataPath + TIMESTAMP + '/recipes_valid_full.pkl')
elif(smallSet == True):
  baseFrame = baseFrame.append(pd.read_pickle(glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl')[0]))
elif(len(glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl')) != 0):
  for file in glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl'):
    if not 'full' in file:
      baseFrame = baseFrame.append(pd.read_pickle(file))

baseFrame.head()

,title,ingredients,instructions
id,,,
000033e39b,Dilly Macaroni Salad Recipe,amount unit ingredient 0 1....,0 Cook macaroni according to package direct...
000035f7ed,Gazpacho,amount unit ingredient 0 8.0 ...,0 Add the tomatoes to a food processor with...
00003a70b1,Crunchy Onion Potato Bake,amount unit ingredient 0 2...,0 Preheat oven to 350 degrees Fah...
00004320bb,Cool 'n Easy Creamy Watermelon Pie,amount unit ingredient 0 3....,0 Dissolve Jello in boiling water. 1 ...
0000631d90,Easy Tropical Beef Skillet,amount unit ingredient 0...,"0 In a large skillet, toast the coconut ove..."


time: 26.2 s (started: 2022-02-20 15:23:56 +00:00)


In [7]:
def flatten_ing(row):
  row['amount'] = row['ingredients']['amount'].tolist()
  row['unit'] = row['ingredients']['unit'].tolist()
  row['ingredient'] = row['ingredients']['ingredient'].tolist()
  return row

baseFrame = baseFrame.apply(flatten_ing, axis=1)
baseFrame = baseFrame.drop(columns=['ingredients'])

baseFrame.head()

,title,instructions,amount,unit,ingredient
id,,,,,
000033e39b,Dilly Macaroni Salad Recipe,0 Cook macaroni according to package direct...,"[1.0, 1.0, 0.5, 0.5, 3.0, 0.5, 1.0, 0.75, 0.5]","[, , , , tablespoon, , tablespoon, teaspoon, t...","[elbow macaroni, american cheese, celery, gree..."
000035f7ed,Gazpacho,0 Add the tomatoes to a food processor with...,"[8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 3.0]","[, , , , , , , , ]","[tomatoes, kosher salt, red onion, green bell ..."
00003a70b1,Crunchy Onion Potato Bake,0 Preheat oven to 350 degrees Fah...,"[2.5, 1.5, 0.25, 1.0, 8.0, 1.0, 1.0]","[, , cup, , ounce, cup, cup]","[milk, water, butter, mashed potatoes, whole k..."
00004320bb,Cool 'n Easy Creamy Watermelon Pie,0 Dissolve Jello in boiling water. 1 ...,"[3.0, 0.25, 12.0, 2.0, 1.0]","[ounce, cup, ounce, cup, ]","[watermelon gelatin, boiling water, cool whip,..."
0000631d90,Easy Tropical Beef Skillet,"0 In a large skillet, toast the coconut ove...","[0.5, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 8.0, 16.0,...","[cup, pound-mass, tablespoon, , tablespoon, ta...","[shredded coconut, lean ground beef, fresh gar..."


time: 2min 58s (started: 2022-02-20 15:24:22 +00:00)


## Brainstorming
* not a simple sequence
* basically sequence of
  * ingredients
  * set of instructions --> instruction --> words
* maybe instead of instructions sequences, instructions could be flattened and split signs (.,!?;) also learned (in case learning sentence sequences is not possible)

Maybe stacked LSTMs are a solutions
* train how instructions in general are to be composed
* train how instructions are combined to a set of instructions

Input
* ingredients to be used as input
* to not train on instructions to appear according to ingredients sequence, maybe randomly shuffle ingredients but keep instruction set the same
* 

https://www.youtube.com/watch?v=A9QVYOBjZdY&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=5&ab_channel=TensorFlow


## Differentiation
* Title generation based on ingredients
* instruction generation

## Import Torch

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.data import get_tokenizer # https://pytorch.org/text/stable/data_utils.html
from torch.utils.data import DataLoader

# torch padding does only support constant padding (ConstantPad1d) for 1D or non-constant padding for >1D (nn.function.pad)
from tensorflow.keras.preprocessing.sequence import pad_sequences
# keras tokenizer more powerful than torch
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence


torch.manual_seed(1)

time: 9.74 s (started: 2022-02-20 15:27:21 +00:00)


In [9]:
# initialize tokenizer
# tokenizer = get_tokenizer("basic_english") # pytorch
tokenizer = Tokenizer() # keras

# to be checked: necessity of punctuation (maybe reintroduce later: https://stackoverflow.com/questions/49073673/include-punctuation-in-keras-tokenizer)

time: 1.58 ms (started: 2022-02-20 15:27:31 +00:00)


# Setup
https://closeheat.com/blog/pytorch-lstm-text-generation-tutorial

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))

Tesla P100-PCIE-16GB
time: 53.8 ms (started: 2022-02-20 15:27:31 +00:00)


### Get Corpus

In [11]:
w2v_model = word2vec.load(dataPath + 'vocab.bin')
ingredientDict = {}
for voc in w2v_model.vocab:
     # Offset by 1 so empty fields can be 0
     ingredientDict.setdefault(voc, len(ingredientDict)+1)

if 'dilly' in ingredientDict:
  print("Word exists")
else:
  print('vocab.bin not to be used as dict misses words') 

vocab.bin not to be used as dict misses words
time: 658 ms (started: 2022-02-20 15:27:31 +00:00)


In [12]:
def getCorpus(title, ingredient, instructions):
  titleTok = text_to_word_sequence(title)
  ingTok = text_to_word_sequence(' '.join(ingredient))
  instTok = text_to_word_sequence(' '.join(instructions))
  return np.array(ingTok + titleTok + instTok)

corpus = np.vectorize(getCorpus, otypes=[np.ndarray])(baseFrame['title'], baseFrame['ingredient'], baseFrame['instructions'])
corpus = np.concatenate(corpus)

time: 6.99 s (started: 2022-02-20 15:27:31 +00:00)


###  Sequences

In [13]:
test = baseFrame.head(20).copy()

def getTitleSequence(title, ingredient):
  titleTok = text_to_word_sequence(title)
  ingTok = text_to_word_sequence(','.join(ingredient))
  return (ingTok + titleTok)

titleSeq = np.vectorize(getTitleSequence, otypes=[np.ndarray])(test['title'], test['ingredient'])
titleSeq[0]

['elbow',
 'macaroni',
 'american',
 'cheese',
 'celery',
 'green',
 'peppers',
 'pimentos',
 'mayonnaise',
 'vinegar',
 'salt',
 'dry',
 'dill',
 'weed',
 'dilly',
 'macaroni',
 'salad',
 'recipe']

time: 7.24 ms (started: 2022-02-20 15:27:38 +00:00)


In [14]:
def getTitleSequence(title, ingredient):
  titleTok = text_to_word_sequence(title)
  ingTok = text_to_word_sequence(','.join(ingredient))
  return {'ings': ingTok, 'title': titleTok}

titleSeq = np.vectorize(getTitleSequence, otypes=[dict])(baseFrame['title'], baseFrame['ingredient'])
titleSeq[0]['ings']

['elbow',
 'macaroni',
 'american',
 'cheese',
 'celery',
 'green',
 'peppers',
 'pimentos',
 'mayonnaise',
 'vinegar',
 'salt',
 'dry',
 'dill',
 'weed']

time: 1.92 s (started: 2022-02-20 15:27:38 +00:00)


In [15]:
def getInstSequence(ingredient, instructions):
  ingTok = text_to_word_sequence(','.join(ingredient))
  instTok = text_to_word_sequence('\n'.join(instructions))
  return np.array(ingTok + instTok)

instSeq = np.vectorize(getInstSequence, otypes=[np.ndarray])(baseFrame['ingredient'], baseFrame['instructions'])
instSeq[0]

array(['elbow', 'macaroni', 'american', 'cheese', 'celery', 'green',
       'peppers', 'pimentos', 'mayonnaise', 'vinegar', 'salt', 'dry',
       'dill', 'weed', 'cook', 'macaroni', 'according', 'to', 'package',
       'directions', 'drain', 'well', 'cold', 'combine', 'macaroni',
       'cheese', 'cubes', 'celery', 'green', 'pepper', 'and', 'pimento',
       'blend', 'together', 'mayonnaise', 'or', 'possibly', 'salad',
       'dressing', 'vinegar', 'salt', 'and', 'dill', 'weed', 'add', 'in',
       'to', 'macaroni', 'mix', 'toss', 'lightly', 'cover', 'and',
       'refrigeratewell', 'serve', 'salad', 'in', 'lettuce', 'lined',
       'bowl', 'if', 'you', 'like', 'makes', '6', 'servings'],
      dtype='<U15')

time: 5.37 s (started: 2022-02-20 15:27:40 +00:00)


### Hyperparams

In [16]:
class hyperParams():
  def __init__(self, epochs=10, batchSize=256, sequenceLength=4):
    self.epochs = epochs
    self.batchSize = batchSize
    self.sequenceLength = sequenceLength

time: 1.61 ms (started: 2022-02-20 15:27:46 +00:00)


### Padding

https://arxiv.org/abs/1903.07288

### Dataset

In [89]:
# Data (https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class)
class TitleDataset(torch.utils.data.Dataset):
    def __init__(self, hyperparams, data):
      self.hyperparams = hyperparams

      self.tokenizer = Tokenizer()

      # dataset split into word sequences required for training
      self.wordSeq = np.vectorize(self.getTitleSequence, otypes=[np.ndarray])(data['title'], data['ingredient'])

      # training requires same length sequences -->  padding
      self.maxSequenceLength = max([max(len(seq['ings']), len(seq['title'])) for seq in self.wordSeq])
      
      # list of all words in dataset
      self.words = np.concatenate(np.vectorize(self.getCorpus, otypes=[np.ndarray])(data['title'], data['ingredient'], data['instructions']))

      # tokenization corpus
      self.tokenizer.fit_on_texts(self.words)

      # indexed wordSequences (could be calculated in getter but very slow, preprocessing better)
      self.idxWords = np.vectorize(self.getIndexedSeqs, otypes=[np.ndarray])(self.wordSeq)


    def getCorpus(self, title, ingredient, instructions):
      titleTok = text_to_word_sequence(title)
      ingTok = text_to_word_sequence(','.join(ingredient))
      instTok = text_to_word_sequence('\n'.join(instructions))
      return np.array(ingTok + titleTok + instTok)

    def getTitleSequence(self, title, ingredient):
      titleTok = text_to_word_sequence(title)
      ingTok = text_to_word_sequence(','.join(ingredient))
      return {'ings': ingTok, 'title': titleTok}

    def getIndexedSeqs(self, seq):
      ingTok = self.tokenizer.texts_to_sequences([seq['ings']])[0]
      ingTok = pad_sequences([ingTok], maxlen=self.maxSequenceLength, padding='pre')[0]
      titleTok = self.tokenizer.texts_to_sequences([seq['title']])[0]
      titleTok = pad_sequences([titleTok], maxlen=self.maxSequenceLength, padding='pre')[0]

      return {'ings': ingTok, 'title': titleTok}

    def __len__(self):
        return len(self.idxWords)

    def __getitem__(self, index):
      # tuple of input (ingredients) and label (title)
        return (
            torch.tensor(self.idxWords[index]['ings'], device=device),
            torch.tensor(self.idxWords[index]['title'], device=device)
        )

time: 32.1 ms (started: 2022-02-20 16:54:55 +00:00)


In [93]:
class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3

        n_vocab = len(dataset.tokenizer.word_index)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

time: 10.6 ms (started: 2022-02-20 17:00:58 +00:00)


In [20]:
def train(dataset, model):
    model.train()

    dataloader = DataLoader(dataset, batch_size=dataset.hyperparams.batchSize)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(dataset.hyperparams.epochs):
        state_h, state_c = model.init_state(dataset.hyperparams.sequenceLength)

        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })

time: 7.5 ms (started: 2022-02-20 15:28:13 +00:00)


In [125]:
def predict(dataset, model, words, next_words=100):
    model.eval()

    state_h, state_c = model.init_state(len(words))
    title = []

    for i in range(0, next_words):
        x = torch.tensor([[dataset.tokenizer.word_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.tokenizer.sequences_to_texts([[word_index]])[0])
        title.append(dataset.tokenizer.sequences_to_texts([[word_index]])[0])

    return title

time: 13.2 ms (started: 2022-02-20 19:24:33 +00:00)


### Execution

In [90]:
titleSet = TitleDataset(hyperParams(), baseFrame)

time: 1min 33s (started: 2022-02-20 16:55:00 +00:00)


In [94]:
model = Model(titleSet)

time: 99.6 ms (started: 2022-02-20 17:01:05 +00:00)


In [97]:
train(titleSet, model)

time: 34.9 s (started: 2022-02-20 17:03:15 +00:00)


In [163]:
idx = 50
print('Orig: ' + str(titleSet.wordSeq[idx]['title']) + '\n' + 
      'Predict: ' + str(predict(titleSet, model, words=titleSet.wordSeq[idx]['ings'], next_words=len(titleSet.wordSeq[idx]['title']))))

Orig: ['hot', 'corned', 'beef', 'buns']
Predict: ['kahmiri', 'machica', 'def', 'brews']
time: 32.2 ms (started: 2022-02-20 19:39:10 +00:00)
